In [2]:
import nltk, re, random

# get started with data and setting
tweets = ([(t, "pos") for t in nltk.corpus.twitter_samples.strings("positive_tweets.json")] +
                            [(t, "neg") for t in nltk.corpus.twitter_samples.strings("negative_tweets.json")])
new_tweets = [re.sub('\||\\n', '', t) for t in nltk.corpus.twitter_samples.strings('tweets.20150430-223406.json')]
stopwords = nltk.corpus.stopwords.words('english')
stemmer = nltk.stem.SnowballStemmer('english')
tokenizer = nltk.tokenize.casual.TweetTokenizer()

# print stats and head of data
n_tweets, n_pos_tweets, n_neg_tweets = len(tweets), len([t for t in tweets if t[1] == "pos"]), len([t for t in tweets if t[1] == "neg"])
print(f"length of tweets: {n_tweets}, positive tweets: {n_pos_tweets}, negative tweets: {n_neg_tweets}\n")
for d in tweets[:6]:
    print(d)

length of tweets: 10000, positive tweets: 5000, negative tweets: 5000

('#FollowFriday @France_Inte @PKuchly57 @Milipol_Paris for being top engaged members in my community this week :)', 'pos')
('@Lamb2ja Hey James! How odd :/ Please call our Contact Centre on 02392441234 and we will be able to assist you :) Many thanks!', 'pos')
('@DespiteOfficial we had a listen last night :) As You Bleed is an amazing track. When are you in Scotland?!', 'pos')
('@97sides CONGRATS :)', 'pos')
('yeaaaah yippppy!!!  my accnt verified rqst has succeed got a blue tick mark on my fb profile :) in 15 days', 'pos')
('@BhaktisBanter @PallaviRuhail This one is irresistible :)\n#FlipkartFashionFriday http://t.co/EbZ0L2VENM', 'pos')


In [3]:
# cleaning all the tweets --> set of words model
def set_of_words(text):
    # remove http links and user references
    text = re.sub('http[s]?://(?:[a-zA-Z]|[0-9]|[$-_@.&+#]|[!*(),]|(?:%[0-9a-fA-F][0-9a-fA-F]))+','', text)
    text = re.sub("(@[A-Za-z0-9_]+)","", text)
    # tokenize
    tokens = tokenizer.tokenize(text)
    # remove stopwords
    tokens = [stemmer.stem(t) for t in tokens if len(t)>1 and not t.isnumeric() and t not in stopwords]
    # set of words
    return {t:1 for t in tokens}

data = [(set_of_words(text), label) for text, label in tweets]
for d in data[:6]:
    print(d)

({'#followfriday': 1, 'top': 1, 'engag': 1, 'member': 1, 'communiti': 1, 'week': 1, ':)': 1}, 'pos')
({'hey': 1, 'jame': 1, 'how': 1, 'odd': 1, ':/': 1, 'pleas': 1, 'call': 1, 'contact': 1, 'centr': 1, 'abl': 1, 'assist': 1, ':)': 1, 'mani': 1, 'thank': 1}, 'pos')
({'listen': 1, 'last': 1, 'night': 1, ':)': 1, 'as': 1, 'you': 1, 'bleed': 1, 'amaz': 1, 'track': 1, 'when': 1, 'scotland': 1}, 'pos')
({'congrat': 1, ':)': 1}, 'pos')
({'yeaaaah': 1, 'yippppi': 1, 'accnt': 1, 'verifi': 1, 'rqst': 1, 'succeed': 1, 'got': 1, 'blue': 1, 'tick': 1, 'mark': 1, 'fb': 1, 'profil': 1, ':)': 1, 'day': 1}, 'pos')
({'this': 1, 'one': 1, 'irresist': 1, ':)': 1, '#flipkartfashionfriday': 1}, 'pos')


In [7]:
# split training and test (stratify pos/neg samples)
pos_data = [x for x in data if x[1] == 'pos']
pos_tweets = [x[0] for x in tweets if x[1] == 'pos']
neg_data = [x for x in data if x[1] == 'neg']
neg_tweets = [x[0] for x in tweets if x[1] == 'neg']

pos_split = 80 * len(pos_data) // 100
neg_split = 80 * len(neg_data) // 100

train_data = pos_data[:pos_split] + neg_data[:neg_split]
train_tweets = pos_tweets[:pos_split] + neg_tweets[:neg_split]
test_data = pos_data[pos_split:] + neg_data[neg_split:]
test_tweets = pos_tweets[pos_split:] + neg_tweets[neg_split:]

In [8]:
# classify with Naive Bayes (bernoulli)
classifier = nltk.NaiveBayesClassifier.train(train_data)

print("Train Accuracy is:", nltk.classify.accuracy(classifier, train_data))
print("Test Accuracy is:", nltk.classify.accuracy(classifier, test_data))
print("\n")
print(classifier.show_most_informative_features(10))

Train Accuracy is: 0.999375
Test Accuracy is: 0.9945


Most Informative Features
                      :( = 1                 neg : pos    =   2362.3 : 1.0
                      :) = 1                 pos : neg    =   1139.0 : 1.0
                     x15 = 1                 neg : pos    =     23.7 : 1.0
                    sick = 1                 neg : pos    =     21.0 : 1.0
               communiti = 1                 pos : neg    =     17.0 : 1.0
                     too = 1                 neg : pos    =     17.0 : 1.0
                     sad = 1                 neg : pos    =     16.5 : 1.0
                     ugh = 1                 neg : pos    =     15.0 : 1.0
                    miss = 1                 neg : pos    =     13.5 : 1.0
                   didnt = 1                 neg : pos    =     13.0 : 1.0
None


In [9]:
from IPython.display import Markdown
from tabulate import tabulate

# get all false predictions
false_predictions = [('train', classifier.classify(t[0]), t[1], re.sub('\||\\n','',train_tweets[i]), ', '.join(t[0].keys())) for i, t in enumerate(train_data) if classifier.classify(t[0]) != t[1]]
false_predictions += [('test', classifier.classify(t[0]), t[1], re.sub('\||\\n','',test_tweets[i]), ', '.join(t[0].keys())) for i, t in enumerate(test_data) if classifier.classify(t[0]) != t[1]]
print(f'false predictions: {len(false_predictions)} out of {len(tweets)}')

headers = ['set', 'predicted', 'actual', 'tweet', 'tokens']
Markdown(tabulate(false_predictions, headers, tablefmt='github'))    

false predictions: 16 out of 10000


| set   | predicted   | actual   | tweet                                                                                                                                    | tokens                                                                                                      |
|-------|-------------|----------|------------------------------------------------------------------------------------------------------------------------------------------|-------------------------------------------------------------------------------------------------------------|
| train | neg         | pos      | Remember that one time I didn't go to flume/kaytranada/alunageorge even though I had tickets? I still want to kms. : ) : )               | rememb, one, time, go, flume, kaytranada, alunageorg, even, though, ticket, still, want, kms                |
| train | neg         | pos      | @ellekagaoan @chinmarquez Catch up once in a while :( &gt;:D&lt; @aditriphosphate @ErinMonzon                                            | catch, :(, >:d                                                                                              |
| train | pos         | neg      | @jackshilling Great news, thanks for letting us know :( We hope you have a good weekend!                                                 | great, news, thank, let, us, know, :(, we, hope, good, weekend                                              |
| train | pos         | neg      | true : ((( https://t.co/hfGyVJQ5RA                                                                                                       | true                                                                                                        |
| train | pos         | neg      | @wtfxmbs AMBS please it's harry's jeans :)):):):(                                                                                        | amb, pleas, harri, jean, :), ):                                                                             |
| test  | neg         | pos      | @jaredNOTsubway @iluvmariah @Bravotv Then that truly is a LATERAL move! Now, we all know the Queen Bee is UPWARD BOUND : ) #MovingOnUp   | then, truli, later, move, now, know, queen, bee, upward, bound, #movingonup                                 |
| test  | neg         | pos      | kik : qualky808 #kik #kikmenow #milf #like4like #bored #summer #sexysaturday :) http://t.co/8R2Nrl31IC                                   | kik, qualki, #kik, #kikmenow, #milf, #like4lik, #bore, #summer, #sexysaturday, :)                           |
| test  | neg         | pos      | A new report talks about how we burn more calories in the cold, because we work harder to warm up. Feel any better about the weather? :p | new, report, talk, burn, calori, cold, work, harder, warm, feel, better, weather, :p                        |
| test  | neg         | pos      | Add me on KIK : ughtmed545 #kik #kikmeguys #kissme #nudes #likeforfollow #musicbiz #sexysasunday :) http://t.co/bgTjjz7fFN               | add, kik, ughtm, #kik, #kikmeguy, #kissm, #nude, #likeforfollow, #musicbiz, #sexysasunday, :)               |
| test  | neg         | pos      | off to the park to get some sunlight : )                                                                                                 | park, get, sunlight                                                                                         |
| test  | neg         | pos      | @msarosh Uff Itna Miss karhy thy ap :p                                                                                                   | uff, itna, miss, karhi, thi, ap, :p                                                                         |
| test  | neg         | pos      | Looking for fun? SNAPCHAT - GooffeAnottere #snapchat #kiksex #snapme #lesbian #instagram #mpoints #mugshot :) http://t.co/p4IqffSWJp     | look, fun, snapchat, gooffeanotter, #snapchat, #kiksex, #snapm, #lesbian, #instagram, #mpoint, #mugshot, :) |
| test  | pos         | neg      | @phenomyoutube u probs had more fun with david than me : (                                                                               | prob, fun, david                                                                                            |
| test  | pos         | neg      | pats jay : (                                                                                                                             | pat, jay                                                                                                    |
| test  | pos         | neg      | @bae_ts WHATEVER STIL L YOUNG &gt;:-(                                                                                                    | whatev, stil, young, >:-(                                                                                   |
| test  | pos         | neg      | my beloved grandmother : ( https://t.co/wt4oXq5xCf                                                                                       | belov, grandmoth                                                                                            |

In [10]:
from tabulate import tabulate
headers = ['predicted', 'tweet']
random.shuffle(new_tweets)
predictions = [classifier.classify(set_of_words(text)) for text in new_tweets[0:10]]
Markdown(tabulate(zip(predictions, new_tweets), headers, tablefmt='github'))

| predicted   | tweet                                                                                                                                        |
|-------------|----------------------------------------------------------------------------------------------------------------------------------------------|
| pos         | RT @TonksterUKIP: A lesson from Nigel Farage                                                                                                 |
| neg         | A writer whose books I quite like appears to be a massive Tory. *disappointed face*                                                          |
| neg         | VIDEO: UKIP 'would accept' EU vote result - http://t.co/MbPFKV38Sb                                                                           |
| pos         | RT @ViscountExmouth: I have never heard a politician answer questions as clearly and directly as @Nigel_Farage #ukip #AskNigelFarage         |
| pos         | Milliband ' s logic.If Tories can get,through deals, X seats and Lab + SNP is X+1 then Milliband will throw vulnerable to Tory wolves. Crap  |
| neg         | Ed Miliband stop wearing a blue tie you pesky kid                                                                                            |
| pos         | Most unfortunate labour dismiss a collaboration with SNP #childish and regressive attitude with the Conservatives the net winners lets see   |
| neg         | RT @Jamin2g: Ed Miliband tanked on the economy, fumbled on a coalition/pact, struggled on immigration, and then stumbled off the stage. #b…  |
| pos         | @IainDale @DouglasCarswell @Nigel_Farage we all know its Tory controlled u.k.I.p                                                             |
| pos         | RT @AndrewMarr9: It's odd that the Tories have no real answer to the benefit cuts charge – and Miliband has no answer to the historic overs… |

In [14]:
from transformers import pipeline

# create pipeline for sentiment analysis
trf_classifier = pipeline('sentiment-analysis', model='cardiffnlp/twitter-roberta-base-sentiment')

In [15]:
from tabulate import tabulate
headers = ['predicted', 'score', 'actual', 'tweet']
predictions = [trf_classifier(text)[0] for text in pos_tweets[0:10]]
rows = [[x['label'], x['score'], 'pos', re.sub('\||\\n','',pos_tweets[i])] for i,x in enumerate(predictions)]
predictions = [trf_classifier(text)[0] for text in neg_tweets[0:10]]
rows += [[x['label'], x['score'], 'neg', re.sub('\||\\n','',neg_tweets[i])] for i,x in enumerate(predictions)]
Markdown(tabulate(rows, headers, tablefmt='github'))

| predicted   |    score | actual   | tweet                                                                                                                                       |
|-------------|----------|----------|---------------------------------------------------------------------------------------------------------------------------------------------|
| LABEL_2     | 0.953848 | pos      | #FollowFriday @France_Inte @PKuchly57 @Milipol_Paris for being top engaged members in my community this week :)                             |
| LABEL_2     | 0.865835 | pos      | @Lamb2ja Hey James! How odd :/ Please call our Contact Centre on 02392441234 and we will be able to assist you :) Many thanks!              |
| LABEL_2     | 0.985131 | pos      | @DespiteOfficial we had a listen last night :) As You Bleed is an amazing track. When are you in Scotland?!                                 |
| LABEL_2     | 0.982928 | pos      | @97sides CONGRATS :)                                                                                                                        |
| LABEL_2     | 0.96674  | pos      | yeaaaah yippppy!!!  my accnt verified rqst has succeed got a blue tick mark on my fb profile :) in 15 days                                  |
| LABEL_2     | 0.961149 | pos      | @BhaktisBanter @PallaviRuhail This one is irresistible :)#FlipkartFashionFriday http://t.co/EbZ0L2VENM                                      |
| LABEL_2     | 0.979076 | pos      | We don't like to keep our lovely customers waiting for long! We hope you enjoy! Happy Friday! - LWWF :) https://t.co/smyYriipxI             |
| LABEL_1     | 0.56171  | pos      | @Impatientraider On second thought, there’s just not enough time for a DD :) But new shorts entering system. Sheep must be buying.          |
| LABEL_1     | 0.600002 | pos      | Jgh , but we have to go to Bayan :D bye                                                                                                     |
| LABEL_1     | 0.616041 | pos      | As an act of mischievousness, am calling the ETL layer of our in-house warehousing app Katamari.Well… as the name implies :p.               |
| LABEL_0     | 0.727188 | neg      | hopeless for tmr :(                                                                                                                         |
| LABEL_2     | 0.923635 | neg      | Everything in the kids section of IKEA is so cute. Shame I'm nearly 19 in 2 months :(                                                       |
| LABEL_0     | 0.92497  | neg      | @Hegelbon That heart sliding into the waste basket. :(                                                                                      |
| LABEL_0     | 0.954726 | neg      | “@ketchBurning: I hate Japanese call him "bani" :( :(”Me too                                                                                |
| LABEL_0     | 0.901618 | neg      | Dang starting next week I have "work" :(                                                                                                    |
| LABEL_0     | 0.708448 | neg      | oh god, my babies' faces :( https://t.co/9fcwGvaki0                                                                                         |
| LABEL_2     | 0.89977  | neg      | @RileyMcDonough make me smile :((                                                                                                           |
| LABEL_0     | 0.844971 | neg      | @f0ggstar @stuartthull work neighbour on motors. Asked why and he said hates the updates on search :( http://t.co/XvmTUikWln                |
| LABEL_1     | 0.640718 | neg      | why?:("@tahuodyy: sialan:( https://t.co/Hv1i0xcrL2"                                                                                         |
| LABEL_0     | 0.741909 | neg      | Athabasca glacier was there in #1948 :-( #athabasca #glacier #jasper #jaspernationalpark #alberta #explorealberta #… http://t.co/dZZdqmf7Cz |

In [16]:
# get all false predictions
pos_predicitions = trf_classifier(pos_tweets[:512])
neg_predicitions = trf_classifier(neg_tweets[:512])

In [93]:
tp = len([x for x in pos_predicitions if x['label']!='LABEL_0'])
fp = len([x for x in pos_predicitions if x['label']=='LABEL_0'])
tn = len([x for x in neg_predicitions if x['label']!='LABEL_2'])
fn = len([x for x in neg_predicitions if x['label']=='LABEL_2'])

print(f'accuracy: {(tp+tn)/(tp+tn+fp+fn)}')

accuracy: 0.9501953125
